# Combine populations for weighted mRS distributions

Use the patient population to combine the mRS distributions from the separate cohorts.

## Plain English summary

In the previous notebook we calculated outcomes for patients across all of England and Wales. This assumed fixed times to treatment except for travel times to the stroke units. We calculated one set of times for the drip-and-ship scenario, where patients first go to their nearest stroke unit and are later transferred to the MT unit if they need thrombectomy, and a second set of times for the mothership scenario, where every patient goes directly to the MT unit.

The existing data has separate entries for each cohort of patients:
+ patients with an nLVO treated with IVT
+ patients with a LVO treated with IVT only
+ patients with a LVO treated with MT only
+ patients with a LVO treated with both IVT and MT

In this notebook we will combine the data of multiple groups of patients.

We will then combine all patients around England and Wales to create one set of mRS distributions that applies nationally.


## Aims

To find the averaged mRS distributions after treatment for the following groups:
+ patients with a LVO and a mix of the available treatment types
+ patients with an nLVO and patients with a LVO and a mix of the available treatment types

Then to combine the mRS distributions for any LSOA where redirection affects which stroke unit is attended first.


## Method

Use the example patient proportions to calculate a weighted sum of the separate cohorts.

The national mRS distribution uses a weighted combination of the mRS distributions of all LSOAs where redirection affects which stroke unit is attended first. The weights are the predicted number of admissions from each LSOA as calculated in a previous notebook.


## Notebook setup

In [1]:
import pandas as pd
import numpy as np
import os

import stroke_outcome.outcome_utilities

In [2]:
dir_data = 'data'
dir_output = 'output'

## Import data

Patient proportions:

In [3]:
proportions = pd.read_csv(
    os.path.join(dir_data, 'patient_proportions.csv'),
    index_col=0, header=None).squeeze()

In [4]:
proportions

0
haemorrhagic         0.13600
lvo_no_treatment     0.14648
lvo_ivt_only         0.00840
lvo_ivt_mt           0.08500
lvo_mt_only          0.01500
nlvo_no_treatment    0.50252
nlvo_ivt             0.10660
Name: 1, dtype: float64

Calculate some additional proportions:

In [5]:
# Proportion of treated LVO patients:
prop_lvo_treated = 0.0

for key, value in proportions.items():
    if (('lvo' in key) & ('nlvo' not in key) & ('no_treat' not in key)):
        print(key)
        prop_lvo_treated += value

prop_lvo_treated

lvo_ivt_only
lvo_ivt_mt
lvo_mt_only


0.10840000000000001

In [6]:
# Proportion of treated ischaemic patients:
prop_ischaemic_treated = 0.0

for key, value in proportions.items():
    if (('lvo' in key) & ('no_treat' not in key)):
        print(key)
        prop_ischaemic_treated += value

prop_ischaemic_treated

lvo_ivt_only
lvo_ivt_mt
lvo_mt_only
nlvo_ivt


0.21500000000000002

In [7]:
# Proportion of ischaemic patients:
prop_ischaemic = 1.0 - proportions['haemorrhagic']

prop_ischaemic

0.864

mRS distributions:

In [8]:
df_mrs = pd.read_csv(os.path.join(dir_output, 'cohort_mrs_dists.csv'), index_col=0)

In [9]:
df_mrs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4537 entries, N00000001 to N00004537
Data columns (total 67 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   closest_ivt_time                   4537 non-null   float64
 1   closest_ivt_unit                   4537 non-null   object 
 2   closest_mt_time                    4537 non-null   float64
 3   closest_mt_unit                    4537 non-null   object 
 4   transfer_mt_time                   4537 non-null   float64
 5   transfer_mt_unit                   4537 non-null   object 
 6   mt_transfer_required               4537 non-null   bool   
 7   ivt_drip_ship                      4537 non-null   float64
 8   mt_drip_ship                       4537 non-null   float64
 9   ivt_mothership                     4537 non-null   float64
 10  mt_mothership                      4537 non-null   float64
 11  drip_ship_nlvo_ivt_mrs_dists_0     4537 non-null

## mRS distributions

### Load in admissions data

In [11]:
admissions = pd.read_csv('../data/admissions_2018.csv')

In [14]:
df_mrs = pd.merge(
    df_mrs.reset_index(), admissions,
    left_on='from_postcode', right_on='area', how='left'
)

### Make non-cumulative mRS distributions

In [15]:
cols_to_combine = [c for c in df_mrs.columns if 'dist' in c]
# Remove repeats from all the mRS bands:
cols_each_scen = sorted(list(set(['_'.join(c.split('_')[:-1]) for c in cols_to_combine])))

In [16]:
cols_each_scen

['drip_ship_lvo_ivt_mrs_dists',
 'drip_ship_lvo_ivt_mt_mrs_dists',
 'drip_ship_lvo_mt_mrs_dists',
 'drip_ship_nlvo_ivt_mrs_dists',
 'mothership_lvo_ivt_mrs_dists',
 'mothership_lvo_ivt_mt_mrs_dists',
 'mothership_lvo_mt_mrs_dists',
 'mothership_nlvo_ivt_mrs_dists']

In [17]:
# Make non-cumulative mRS distributions:
for c in cols_each_scen:
    cols_cumsum = [f'{c}_{i}' for i in range(7)]
    cols_noncum = [f'{c}_noncum_{i}' for i in range(7)]

    new_data = df_mrs[cols_cumsum].copy()
    # Take the difference between mRS bands:
    new_data = np.diff(new_data, prepend=0.0, axis=1)
    # Round the values:
    new_data = np.round(new_data, 5)
    # Store:
    df_mrs[cols_noncum] = new_data

### Import reference no-treatment data

In [18]:
mrs_dists, mrs_dists_notes = (
    stroke_outcome.outcome_utilities.import_mrs_dists_from_file())

In [19]:
mrs_dists

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
Stroke type,,,,,,,
pre_stroke_nlvo,0.583,0.746,0.850,0.951,0.993,1.000,1
pre_stroke_lvo,0.408,0.552,0.672,0.838,0.956,1.000,1
no_treatment_lvo,0.050,0.129,0.265,0.429,0.676,0.811,1
no_treatment_nlvo,0.198,0.460,0.580,0.708,0.856,0.918,1
no_effect_nlvo_ivt_deaths,0.196,0.455,0.574,0.701,0.847,0.908,1
no_effect_lvo_ivt_deaths,0.048,0.124,0.255,0.414,0.653,0.783,1
no_effect_lvo_mt_deaths,0.048,0.124,0.255,0.412,0.649,0.779,1
t0_treatment_nlvo_ivt,0.445,0.642,0.752,0.862,0.941,0.967,1
t0_treatment_lvo_ivt,0.140,0.233,0.361,0.522,0.730,0.838,1


In [20]:
mrs_dist_nlvo_no_treatment = mrs_dists.loc['no_treatment_nlvo'].values
mrs_dist_lvo_no_treatment = mrs_dists.loc['no_treatment_lvo'].values

mrs_dist_nlvo_no_treatment_noncum = np.diff(mrs_dist_nlvo_no_treatment, prepend=0.0)
mrs_dist_lvo_no_treatment_noncum = np.diff(mrs_dist_lvo_no_treatment, prepend=0.0)

### Combine patient cohorts

Combine LVO groups:

In [21]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_lvo_mix_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_lvo_mix_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only'])
    )
    # Remove the proportion of non-LVO strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_lvo_treated
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)

    # Round the results:
    df_mrs[cols_noncum] = np.round(df_mrs[cols_noncum], 5)
    df_mrs[cols_cum] = np.round(df_mrs[cols_cum], 5)

Weighted _treated_ ischaemic population.

We have no mRS distributions for the haemorrhagic strokes. We calculate the weighted population mRS distributions using the data that we do have, and then divide by the proportion of the population that is not haemorrhagic. This rescales the proportions so that all proportions excluding haemorrhagic sum to 1.

In [22]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_weighted_treated_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_weighted_treated_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_nlvo_ivt = [f'{model_name}_nlvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_nlvo_ivt].values * proportions['nlvo_ivt']) +
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only'])
    )
    # Remove the proportion of haemorrhagic strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_ischaemic_treated
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)
    
    # Round the results:
    df_mrs[cols_noncum] = np.round(df_mrs[cols_noncum], 5)
    df_mrs[cols_cum] = np.round(df_mrs[cols_cum], 5)

Weighted _total_ ischaemic population.

We have no mRS distributions for the haemorrhagic strokes. We calculate the weighted population mRS distributions using the data that we do have, and then divide by the proportion of the population that is not haemorrhagic. This rescales the proportions so that all proportions excluding haemorrhagic sum to 1.

In [23]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_weighted_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_weighted_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_nlvo_ivt = [f'{model_name}_nlvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_nlvo_ivt].values * proportions['nlvo_ivt']) +
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only']) +
        (mrs_dist_nlvo_no_treatment_noncum * proportions['nlvo_no_treatment']) +
        (mrs_dist_lvo_no_treatment_noncum * proportions['lvo_no_treatment'])
    )
    # Remove the proportion of haemorrhagic strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_ischaemic
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)
    
    # Round the results:
    df_mrs[cols_noncum] = np.round(df_mrs[cols_noncum], 5)
    df_mrs[cols_cum] = np.round(df_mrs[cols_cum], 5)

### Average mRS distributions by admissions

In [24]:
# Remove repeats from all the mRS bands:
cols_each_scen = sorted(list(set(
    [c.split('mrs_dist')[0] + 'mrs_dist' for c in df_mrs.columns if 'mrs_dist' in c]
)))

In [25]:
cols_each_scen

['drip_ship_lvo_ivt_mrs_dist',
 'drip_ship_lvo_ivt_mt_mrs_dist',
 'drip_ship_lvo_mix_mrs_dist',
 'drip_ship_lvo_mt_mrs_dist',
 'drip_ship_nlvo_ivt_mrs_dist',
 'drip_ship_weighted_mrs_dist',
 'drip_ship_weighted_treated_mrs_dist',
 'mothership_lvo_ivt_mrs_dist',
 'mothership_lvo_ivt_mt_mrs_dist',
 'mothership_lvo_mix_mrs_dist',
 'mothership_lvo_mt_mrs_dist',
 'mothership_nlvo_ivt_mrs_dist',
 'mothership_weighted_mrs_dist',
 'mothership_weighted_treated_mrs_dist']

Only make national statistics for areas where mothership and drip and ship are different:

In [26]:
mask = df_mrs['mt_transfer_required']

In [27]:
from statsmodels.stats.weightstats import DescrStatsW  # for mRS dist stats

In [28]:
df_mrs_national = pd.DataFrame()
df_mrs_national_noncum = pd.DataFrame()
df_mrs_national_std = pd.DataFrame()

# Stats:
for c, col in enumerate(cols_each_scen):
    cols_here = [c for c in df_mrs.columns if ((c.startswith(col)) & ('noncum' in c))]

    # Split list of values into one column per mRS band
    # and keep one row per LSOA.
    vals = df_mrs.loc[mask, cols_here].copy()

    # Create stats from these data:
    weighted_stats = DescrStatsW(vals, weights=df_mrs.loc[mask, 'Admissions'], ddof=0)
    # Means (one value per mRS):
    means = weighted_stats.mean
    # Standard deviations (one value per mRS):
    stds = weighted_stats.std
    # Cumulative probability from the mean bins:
    cumulatives = np.cumsum(means)
    
    # Store the data in these columns:
    cols_cumulative = [f'mRS<={i}' for i in range(7)]
    cols_noncum = [f'mRS={i}' for i in range(7)]

    # Store in results df. Round the values to 5 decimal places.
    val = col.split('_mrs_dist')[0]
    df_mrs_national_noncum.loc[val, cols_noncum] = np.round(means, 5)
    df_mrs_national.loc[val, cols_cumulative] = np.round(cumulatives, 5)
    df_mrs_national_std.loc[val, cols_noncum] = np.round(stds, 5)

In [29]:
df_mrs_national

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
drip_ship_lvo_ivt,0.10295,0.19476,0.32724,0.48949,0.70813,0.82285,1.0
drip_ship_lvo_ivt_mt,0.11199,0.22184,0.36699,0.53922,0.74661,0.84823,1.0
drip_ship_lvo_mix,0.11120,0.21968,0.36391,0.53540,0.74367,0.84629,1.0
drip_ship_lvo_mt,0.11129,0.22140,0.36697,0.53948,0.74691,0.84840,1.0
drip_ship_nlvo_ivt,0.35977,0.58797,0.70409,0.82330,0.92076,0.95487,1.0
drip_ship_weighted,0.18198,0.38952,0.51479,0.65327,0.81938,0.89541,1.0
drip_ship_weighted_treated,0.23444,0.40228,0.53257,0.67815,0.83147,0.90012,1.0
mothership_lvo_ivt,0.09454,0.18518,0.31825,0.48052,0.70190,0.81845,1.0
mothership_lvo_ivt_mt,0.14982,0.27018,0.41466,0.58775,0.77934,0.86991,1.0
mothership_lvo_mix,0.14553,0.26359,0.40718,0.57944,0.77334,0.86592,1.0


In [30]:
df_mrs_national_noncum

,mRS=0,mRS=1,mRS=2,mRS=3,mRS=4,mRS=5,mRS=6
drip_ship_lvo_ivt,0.10295,0.09181,0.13249,0.16225,0.21864,0.11472,0.17715
drip_ship_lvo_ivt_mt,0.11199,0.10984,0.14515,0.17223,0.20739,0.10162,0.15177
drip_ship_lvo_mix,0.11120,0.10848,0.14423,0.17149,0.20827,0.10262,0.15371
drip_ship_lvo_mt,0.11129,0.11011,0.14557,0.17251,0.20743,0.10149,0.15160
drip_ship_nlvo_ivt,0.35977,0.22820,0.11612,0.11922,0.09746,0.03411,0.04513
drip_ship_weighted,0.18198,0.20754,0.12527,0.13848,0.16611,0.07603,0.10459
drip_ship_weighted_treated,0.23444,0.16784,0.13029,0.14557,0.15333,0.06865,0.09988
mothership_lvo_ivt,0.09454,0.09064,0.13307,0.16227,0.22137,0.11656,0.18155
mothership_lvo_ivt_mt,0.14982,0.12036,0.14448,0.17309,0.19159,0.09057,0.13009
mothership_lvo_mix,0.14553,0.11805,0.14360,0.17226,0.19390,0.09258,0.13408


In [31]:
df_mrs_national_std

,mRS=0,mRS=1,mRS=2,mRS=3,mRS=4,mRS=5,mRS=6
drip_ship_lvo_ivt,0.00362,0.00040,0.00031,0.00005,0.00114,0.00075,0.00177
drip_ship_lvo_ivt_mt,0.01256,0.00583,0.00262,0.00226,0.00604,0.00503,0.01025
drip_ship_lvo_mix,0.01183,0.00527,0.00214,0.00192,0.00563,0.00461,0.00945
drip_ship_lvo_mt,0.01379,0.00506,0.00080,0.00115,0.00614,0.00467,0.00980
drip_ship_nlvo_ivt,0.00922,0.00297,0.00054,0.00088,0.00221,0.00103,0.00159
drip_ship_weighted,0.00222,0.00060,0.00025,0.00022,0.00086,0.00065,0.00129
drip_ship_weighted_treated,0.00891,0.00239,0.00102,0.00089,0.00347,0.00260,0.00518
mothership_lvo_ivt,0.00686,0.00110,0.00041,0.00005,0.00227,0.00155,0.00375
mothership_lvo_ivt_mt,0.01480,0.00306,0.00109,0.00036,0.00575,0.00372,0.00702
mothership_lvo_mix,0.01418,0.00291,0.00104,0.00033,0.00548,0.00355,0.00677


## Errors for weighted combinations

Redo the standard deviations to properly propagate the errors from the separate cohorts.

LVO mix:

In [32]:
drip_ship_lvo_stds_before = df_mrs_national_std.loc['drip_ship_lvo_mix'].copy()
mothership_lvo_stds_before = df_mrs_national_std.loc['mothership_lvo_mix'].copy()

In [33]:
stds_to_combine = ['lvo_ivt', 'lvo_mt', 'lvo_ivt_mt']

for model_name in ['drip_ship', 'mothership']:
    # New column names for std:
    ind = f'{model_name}_lvo_mix'

    # Column names of existing data:
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national_std.loc[ind] = np.sqrt(
        (df_mrs_national_std.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only']  / prop_lvo_treated))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt']  / prop_lvo_treated))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_mt].values * (proportions['lvo_mt_only']  / prop_lvo_treated))**2.0
    )

    # Round to 5 decimal places:
    df_mrs_national_std.loc[ind] = np.round(df_mrs_national_std.loc[ind], 5)

In [34]:
pd.DataFrame(
    np.stack((
        drip_ship_lvo_stds_before,
        df_mrs_national_std.loc['drip_ship_lvo_mix'],
        mothership_lvo_stds_before,
        df_mrs_national_std.loc['mothership_lvo_mix']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.01183,0.01004,0.01418,0.01180
1,0.00527,0.00462,0.00291,0.00244
2,0.00214,0.00206,0.00104,0.00087
3,0.00192,0.00178,0.00033,0.00029
4,0.00563,0.00481,0.00548,0.00458
5,0.00461,0.00400,0.00355,0.00296
6,0.00945,0.00815,0.00677,0.00560


Treated population mix:

In [35]:
drip_ship_weighted_treated_stds_before = df_mrs_national_std.loc['drip_ship_weighted_treated'].copy()
mothership_weighted_treated_stds_before = df_mrs_national_std.loc['mothership_weighted_treated'].copy()

In [36]:
p = prop_ischaemic_treated

for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    ind = f'{model_name}_weighted_treated'

    # Column names of existing data:
    ind_nlvo_ivt = f'{model_name}_nlvo_ivt'
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national_std.loc[ind] = np.sqrt(
        (df_mrs_national_std.loc[ind_nlvo_ivt].values * (proportions['nlvo_ivt'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_mt].values * (proportions['lvo_mt_only'] / p))**2.0
    )
    
    # Round to 5 decimal places:
    df_mrs_national_std.loc[ind] = np.round(df_mrs_national_std.loc[ind], 5)

In [37]:
pd.DataFrame(
    np.stack((
        drip_ship_weighted_treated_stds_before,
        df_mrs_national_std.loc['drip_ship_weighted_treated'],
        mothership_weighted_treated_stds_before,
        df_mrs_national_std.loc['mothership_weighted_treated']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)    

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.00891,0.00682,0.01627,0.01089
1,0.00239,0.00276,0.00122,0.00292
2,0.00102,0.00107,0.00097,0.00063
3,0.00089,0.00100,0.00093,0.00080
4,0.00347,0.00266,0.00509,0.00328
5,0.00260,0.00208,0.00292,0.00187
6,0.00518,0.00419,0.00521,0.00334


Total ischaemic population mix:

In [38]:
drip_ship_weighted_stds_before = df_mrs_national_std.loc['drip_ship_weighted'].copy()
mothership_weighted_stds_before = df_mrs_national_std.loc['mothership_weighted'].copy()

In [39]:
p = prop_ischaemic

for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    ind = f'{model_name}_weighted'

    # Column names of existing data:
    ind_nlvo_ivt = f'{model_name}_nlvo_ivt'
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national_std.loc[ind] = np.sqrt(
        (df_mrs_national_std.loc[ind_nlvo_ivt].values * (proportions['nlvo_ivt'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt'] / p))**2.0 +
        (df_mrs_national_std.loc[ind_lvo_mt].values * (proportions['lvo_mt_only'] / p))**2.0
    )
    
    # Round to 5 decimal places:
    df_mrs_national_std.loc[ind] = np.round(df_mrs_national_std.loc[ind], 5)

In [40]:
pd.DataFrame(
    np.stack((
        drip_ship_weighted_stds_before,
        df_mrs_national_std.loc['drip_ship_weighted'],
        mothership_weighted_stds_before,
        df_mrs_national_std.loc['mothership_weighted']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)    

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.00222,0.00170,0.00405,0.00271
1,0.00060,0.00069,0.00030,0.00073
2,0.00025,0.00027,0.00024,0.00016
3,0.00022,0.00025,0.00023,0.00020
4,0.00086,0.00066,0.00127,0.00082
5,0.00065,0.00052,0.00073,0.00047
6,0.00129,0.00104,0.00130,0.00083


## Save results

In [41]:
# Save to file:
df_mrs.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted.csv'))

In [42]:
# Save to file:
df_mrs_national.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national.csv'))
df_mrs_national_noncum.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national_noncum.csv'))
df_mrs_national_std.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national_std.csv'))

## Conclusion

We have calculated the mRS distributions for mixed cohorts of patients and for the average across England and Wales.